# Intro to Recurrent Neural Networks: Character-wise RNN Walkthrough

Following along the content in DLND

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we load the text file and convert it into integers for network to use

In [2]:
with open('anna.txt', 'r') as f:
    text = f.read()
vocab = set(text)
vocab2int = {c: i for i, c in enumerate(vocab)}
int2vocab = dict(enumerate(vocab))
chars = np.array([vocab2int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([13, 45, 63, 58, 12, 68, 23,  8, 24, 39, 39, 39, 51, 63, 58, 58,  2,
        8, 75, 63, 80, 32,  5, 32, 68, 44,  8, 63, 23, 68,  8, 63,  5,  5,
        8, 63,  5, 32, 69, 68, 77,  8, 68,  4, 68, 23,  2,  8, 70, 20, 45,
       63, 58, 58,  2,  8, 75, 63, 80, 32,  5,  2,  8, 32, 44,  8, 70, 20,
       45, 63, 58, 58,  2,  8, 32, 20,  8, 32, 12, 44,  8, 57, 73, 20, 39,
       73, 63,  2, 76, 39, 39, 49,  4, 68, 23,  2, 12, 45, 32, 20], dtype=int32)

Now need to split up the date into batches, and into training and validation sets. Should be making a test set here, but the test will be if the network can generate new text.

Make both input and target arrays. The targets are the same as outputs, except shifted one character over. Also drop the last bit of data so that there are only completely full batches.

The idea is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. Split this data into a training set and validation set using the split_frac keyword. This will keep 90% 

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ----------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches * slice_size]
    y = chars[1: n_batches * slice_size + 1]
    
    # Split the data into batch-size slices, then stack them into a 2D matrix
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch-size x n_batches * num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches * split_frac)
    train_x, train_y = x[:, :split_idx * num_steps], y[:, :split_idx * num_steps]
    val_x, val_y = x[:, split_idx * num_steps:], y[:, split_idx * num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:, :10]

array([[13, 45, 63, 58, 12, 68, 23,  8, 24, 39],
       [55, 20, 62,  8, 45, 68,  8, 80, 57,  4],
       [ 8,  9, 63, 12,  9, 45, 32, 20, 42,  8],
       [57, 12, 45, 68, 23,  8, 73, 57, 70,  5],
       [ 8, 12, 45, 68,  8,  5, 63, 20, 62, 31],
       [ 8, 26, 45, 23, 57, 70, 42, 45,  8,  5],
       [12,  8, 12, 57, 39, 62, 57, 76, 39, 39],
       [57,  8, 45, 68, 23, 44, 68,  5, 75, 11],
       [45, 63, 12,  8, 32, 44,  8, 12, 45, 68],
       [68, 23, 44, 68,  5, 75,  8, 63, 20, 62]], dtype=int32)

Another function to grab batches out of arrays made by split data. Each batch will be a sliding window on these arrays with size batch_size X num_steps. For example, if we want our network to train on a sequence of 100 characters, num_steps = 100. For the next batch, we'll shift this window the next sequence of num_steps characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size / num_steps)
    for b in range(n_batches):
        yield [x[:, b * num_steps: (b + 1) * num_steps] for x in arrs]
    

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
             learning_rate=0.001, grad_clip=5, sampling=False):
    
    if sampling == True:
        batch_size, num_steps = 1, 1
        
    tf.reset_default_graph()
    
    # Declare placeholders we will feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input characters
    x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
    
    # Build the RNN layers
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    #cell = tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    # Run the data through the RNN layers
    #rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    #outputs, state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state=initial_state)
    outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    final_state = state
    
    # Reshape output so its a bunch of rows, one row for each cell output
    seq_output = tf.concat(outputs, axis=1, name='seq_output')
    output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN outputs to a softmax layer and calculate the cost
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1), name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        
    logits = tf.matmul(output, softmax_w) + softmax_b
    preds = tf.nn.softmax(logits, name='predictions')
    
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
    cost = tf.reduce_mean(loss, name='cost')
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state', 
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

# Hyperparameters 
Defining the hyperparameters for the network. lstm_size and num_layers set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but have to watch out for overfitting. If validation loss is much larger than the training loss, it is probably overfitting. Decrease the size of the network or decrease the dropout keep probability

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Write out graph for TensorBoard

In [12]:
model = build_rnn(len(vocab), 
                 batch_size=batch_size,
                 num_steps=num_steps,
                 learning_rate=learning_rate,
                 lstm_size=lstm_size,
                 num_layers=num_layers)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./logs/1', sess.graph)

ValueError: Trying to share variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel, but specified shape (1024, 2048) and found shape (595, 2048).

# Training
Time for training which is pretty straightforward. Pass in some data, and get an LSTM state back.  Then pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by save_every_n) calculate the validation loss and save a checkpoint.

In [18]:
!mkdir -p checkpoints/anna

In [19]:
epochs = 20
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab),
                 batch_size=batch_size,
                 num_steps=num_steps,
                 learning_rate=learning_rate,
                 lstm_size=lstm_size,
                 num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x,y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e * n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                   model.targets: y,
                   model.keep_prob: 0.5,
                   model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer],
                                               feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{}'.format(e+1, epochs),
                 'Iteration {}/{}'.format(iteration, iterations),
                 'Training loss: {:.4f}'.format(loss/b),
                 '{:.4f} sec / batch'.format((end-start)))
            
            if (iteration % save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x,y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                           model.targets: y,
                           model.keep_prob: 1.,
                           model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state],
                                                    feed_dict=feed)
                    val_loss.append(batch_loss)
                    
                print('Validation loss', np.mean(val_loss), 'Saving checkpoint!')
                saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))
                
            

ValueError: Trying to share variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights, but specified shape (1024, 2048) and found shape (595, 2048).

# Sampling
Now that the network is trained, we can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. Mat also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, a new character is going to be chose from the top N most likely characters.

In [23]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [25]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1,1))
            x[0,0] = vocab2int[c]
            feed = {model.inputs: x,
                   model.keep_prob: 1.,
                   model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state],
                                       feed_dict=feed)
            
        c = pick_top_n(preds, len(vocab))
        samples.append(int2vocab[c])
        
        for i in range(n_samples):
            x[0, 0] = c
            feed = {model.inputs: x,
                   model.keep_prob: 1.,
                   model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state],
                                        feed_dict=dict)
            c = pick_top_n(preds, len(vocab))
            samples.append(int2vocabvocab[c])
            
        return ''.join(samples)

In [26]:
checkpoint = 'checkpoints/anna/i3560_1512_1.122.ckpt'
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

ValueError: Trying to share variable rnn/multi_rnn_cell/cell_0/basic_lstm_cell/weights, but specified shape (1024, 2048) and found shape (595, 2048).